In [ ]:
import numpy as np

from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage import data
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import sys, subprocess

In [ ]:
import cv2
file_path = 'Data/20200212_150024.jpg'
img = cv2.imread(file_path)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [ ]:
plt.figure(figsize = (20,20))
plt.imshow(gray , cmap = 'gray')

In [ ]:
import skimage
from skimage.measure import label, regionprops
from skimage.color import label2rgb
im = Image.open('Data/20200212_150024.jpg')
gray = skimage.color.rgb2gray(np.array(im))
imageTH = gray < .4
selem = skimage.morphology.disk(5)
openImage = skimage.morphology.opening(imageTH, selem)
imageTHCorr = np.logical_and( np.logical_not(openImage), imageTH)
label_image = label(imageTHCorr)

In [ ]:
plt.imshow(imageTHCorr, cmap = 'gray')

In [ ]:
lines = skimage.transform.probabilistic_hough_line(imageTHCorr, threshold=1, line_length=150,
                                 line_gap=5)

In [ ]:
# Generating figure 2
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(gray, cmap='gray')
ax[0].set_title('Input image')

ax[1].imshow(imageTHCorr, cmap='gray')
ax[1].set_title('Canny edges')

ax[2].imshow(imageTHCorr * 0)
for line in lines:
    p0, p1 = line
    ax[2].plot((p0[0], p1[0]), (p0[1], p1[1]))
ax[2].set_xlim((0, imageTHCorr.shape[1]))
ax[2].set_ylim((imageTHCorr.shape[0], 0))
ax[2].set_title('Probabilistic Hough')

for a in ax:
    a.set_axis_off()

plt.tight_layout()

In [ ]:
plt.figure(figsize = (20, 20))
plt.imshow(imageTHCorr * 0)
for line in lines:
    p0, p1 = line
    plt.plot((p0[0], p1[0]), (p0[1], p1[1]))
plt.gca().set_xlim((0, imageTHCorr.shape[1]))
plt.gca().set_ylim((imageTHCorr.shape[0], 0))
plt.gca().set_title('Probabilistic Hough')


In [ ]:
def distAngle(theta, theta_ref):
    v = [np.abs(theta - theta_ref), np.abs(- 2 *np.pi +  theta - theta_ref), np.abs(2 *np.pi +  theta - theta_ref)]
    return min(v)

In [ ]:
#Assume they are relatively near 0 and pi/2m
rhos0 = []
rhospiHalfs = []
for l in lines:
    v = np.array(l[0]) - np.array(l[1])
    angle = np.angle(v[0] + 1j * v[1])
    if distAngle(angle, 0) < 0.1: 
        rhos0.append(angle if angle < np.pi else angle - 2 * np.pi)
    elif distAngle(angle, np.pi) < 0.1:
        rhos0.append(angle - np.pi)
        
    if distAngle(angle, np.pi/2) < 0.1: 
        rhospiHalfs.append(angle)
    elif distAngle(angle, np.pi *3/2) < 0.1:
        rhospiHalfs.append(angle - np.pi)
r0 = np.mean(rhos0)
v0 = np.array([np.cos(r0), np.sin(r0)])
rpiHalf = np.mean(rhospiHalfs)
vpiHalf = np.array([np.cos(rpiHalf), np.sin(rpiHalf)])


In [ ]:
def _line2line(line1, line2):

    """
    - line1 is a list of two xy tuples
    - line2 is a list of two xy tuples
    References consulted:
    http://mathforum.org/library/drmath/view/51980.html
    and http://mathforum.org/library/drmath/view/51926.html
    and https://answers.yahoo.com/question/index?qid=20110507163534AAgvfQF
    """

    import math
    #step1: cross prod the two lines to find common perp vector
    (L1x1,L1y1),(L1x2,L1y2) = line1
    (L2x1,L2y1),(L2x2,L2y2) = line2
    L1dx,L1dy = L1x2-L1x1,L1y2-L1y1
    L2dx,L2dy = L2x2-L2x1,L2y2-L2y1
    commonperp_dx,commonperp_dy = (L1dy - L2dy, L2dx-L1dx)

    #step2: normalized_perp = perp vector / distance of common perp
    commonperp_length = math.hypot(commonperp_dx,commonperp_dy)
    commonperp_normalized_dx = commonperp_dx/float(commonperp_length)
    commonperp_normalized_dy = commonperp_dy/float(commonperp_length)

    #step3: length of (pointonline1-pointonline2 dotprod normalized_perp).
    # Note: According to the first link above, it's sufficient to
    #    "Take any point m on line 1 and any point n on line 2."
    #    Here I chose the startpoint of both lines
    shortestvector_dx = (L1x1-L2x1)*commonperp_normalized_dx
    shortestvector_dy = (L1y1-L2y1)*commonperp_normalized_dy
    mindist = math.hypot(shortestvector_dx,shortestvector_dy)

    #return results
    result = mindist
    return result

In [ ]:
def _line2line(line1, line2, v):
    a, b, c, d = line1[0].dot(v), line1[1].dot(v), line2[0].dot(v), line2[1].dot(v)
    if a > b:
        a,b = b, a
    if c > d:
        c, d = d, c
    return max(max(a,c) -min(b,d), 0)

In [ ]:
np.dot(v0, vpiHalf)
#TODO: projective transform, using a chessboard

In [ ]:
l

In [ ]:
plt.figure(figsize = (20, 20))
#For each line, check if they are approximately r0 or piHalf
#Repetitive step, could reuse previous information
horizontalLines = []
verticalLines = []
for l in lines:
    p0, p1 = np.array(l[0]), np.array(l[1])
    v = (np.array(l[0]) - np.array(l[1])).astype(float)
    v /= np.linalg.norm(v)
    
    #manual crop
    if p0[0] < 500 or p0[0] > 3500:
        continue

    if np.abs(np.dot(v, v0)) > 1 -1e-2:
        b = np.dot(vpiHalf, p0/2 + p1/2)
        horizontalLines.append([p0, p1, b])
    elif np.abs(np.dot(v, vpiHalf)) > 1- 1e-2:
        b = np.dot(v0, p0/2 + p1/2)
        verticalLines.append([p0, p1, b])

    else:
        continue
    plt.plot((p0[0], p1[0]), (p0[1], p1[1]))
plt.gca().set_xlim((0, imageTHCorr.shape[1]))
plt.gca().set_ylim((imageTHCorr.shape[0], 0))
plt.gca().set_title('Probabilistic Hough cropped')


In [ ]:
class Labels:
    def __init__(self, n):
        self.labels = {i : [i] for i in range(n)}
        self.labelIds = {i : i for i in range(n)}
    def mergeLabels(self, i, j):
        li_old = self.labelIds[i]
        lj = self.labelIds[j]
        if li_old == lj:
            return
        for k in self.labels[li_old]:
            self.labelIds[k] = self.labelIds[j]
        self.labels[lj] += self.labels[li_old]
        del self.labels[li_old]

In [ ]:
v = [174, 261, 57, 27, 167, 231, 476, 496, 513, 530, 544, 564, 581, 645]
#Join lines if b difference is small, and in axis is smaller than 300 pixels (to adjust)
horizontalLines = np.array(horizontalLines)
labelsHorizontal = Labels(len(horizontalLines))
#Force that any can have only one neighbour
for i,j in itertools.product(range(len(horizontalLines)), repeat = 2):
    if i <= j:
        continue
    if np.abs(horizontalLines[i][2] - horizontalLines[j][2]) < 10 and  \
            _line2line(horizontalLines[i][:2], horizontalLines[j][:2], v0) < 50:
        labelsHorizontal.mergeLabels(i,j)


In [ ]:
def flatten2(v):
    vRes = np.zeros((len(v) * 2, 2))
    for i,vv in enumerate(v):
        vRes[2* i, :] = vv[0]
        vRes[2* i + 1, :] = vv[1]
    return vRes

In [ ]:
plt.figure(figsize = (20, 20))
for l in verticalLines:
    p0, p1, _ = l
    plt.plot((p0[0], p1[0]), (p0[1], p1[1]))
for l,v  in labelsHorizontal.labels.items():
    points = flatten2(horizontalLines[np.array(v)][:, :2])
    pointsDotV = points.dot(v0)
    i,j = np.argmin(pointsDotV), np.argmax(pointsDotV) #Substitute by a curve fitting -> more stable
    plt.plot((points[i,0], points[j,0]), (points[i,1], points[j,1]), alpha = .5)
    #plt.scatter(points[:,0], points[:,1])

plt.gca().set_xlim((0, imageTHCorr.shape[1]))
plt.gca().set_ylim((imageTHCorr.shape[0], 0))


In [ ]:
print(horizontalLines[261])

In [ ]:
print(horizontalLines[174])